<a href="https://colab.research.google.com/github/c-marq/CAP3321C-Data-Wrangling/blob/main/exercises/chapter-06/exercise_6_1_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 6-1: Clean the Polling Data

## 🔑 INSTRUCTOR SOLUTION KEY

**CAP3321C - Data Wrangling**

---

## Overview

This exercise will guide you through the process of cleaning the Polling data for the 2016 election for president of the United States. As you clean that data, you'll use most of the procedures and methods presented in this chapter.

**Instructions:**
1. Run the setup cells to load the data
2. Complete each task by writing code in the provided cells
3. Some tasks are pre-filled - just run them and observe
4. Tasks marked with **YOUR CODE** require you to write the code

**Group Members:**
- SOLUTION KEY

---

## Read the Data

Run these cells to load the data. Do not modify this section.

In [ ]:
import pandas as pd

In [ ]:
# Download the data file from GitHub
!wget -q https://raw.githubusercontent.com/c-marq/CAP3321C-Data-Wrangling/main/data/president_polls_2016.csv
print("Data file downloaded successfully!")

In [ ]:
# Load the polling data
polls = pd.read_csv('president_polls_2016.csv')
print("Data shape:", polls.shape)
polls.head()

---

## Part 1: Examine the Data

Before cleaning data, you need to understand what you're working with.

### Task 4: Run the info() Method (YOUR CODE)

Run the `info()` method on the DataFrame. Make a note of the columns that have missing values.

**Hint:** Columns with missing values will show fewer non-null counts than the total number of rows.

**Expected output:** A summary showing column names, non-null counts, and data types

In [ ]:
# ✅ SOLUTION - run info() on the polls DataFrame
polls.info()

# Note: Columns with missing values include:
# - grade (some pollsters don't have grades)
# - rawpoll_mcmullin and adjpoll_mcmullin (McMullin wasn't in all polls)

#### 📝 Instructor Notes - Task 4

**Key Teaching Points:**
- `info()` shows non-null count vs total entries to identify missing data
- Data types shown as `object` usually mean strings (need conversion)
- Memory usage shown at bottom

**Columns with missing values:**
- `grade` - not all pollsters have ratings
- `rawpoll_mcmullin` / `adjpoll_mcmullin` - McMullin only on ballot in some states

### Task 5: Run the nunique() Method (YOUR CODE)

Run the `nunique()` method on the DataFrame. Make a note of the columns with only one value.

**Hint:** Columns with only 1 unique value don't provide useful information for analysis.

**Expected output:** A Series showing the count of unique values for each column

In [ ]:
# ✅ SOLUTION - run nunique() on the polls DataFrame
polls.nunique()

# Note: Columns with only 1 unique value:
# - cycle (all 2016)
# - branch (all President)
# - matchup (all same matchup)

#### 📝 Instructor Notes - Task 5

**Key Teaching Points:**
- `nunique()` helps identify columns that don't vary (useless for analysis)
- Columns with 1 value are candidates for removal
- Also useful for identifying categorical vs continuous data

---

## Part 2: Drop Columns and Rows

Remove unnecessary data to focus on what matters.

### Task 6: Filter Rows by Type (YOUR CODE)

Drop all rows **except** where the `type` column contains a value of `"now-cast"`.

**Hint:** Use `.query()` to filter rows, then reassign to `polls`.

**Example syntax:**
```python
polls = polls.query('column == "value"')
```

**Expected output:** The DataFrame should have fewer rows (only "now-cast" type polls)

In [ ]:
# ✅ SOLUTION - keep only rows where type == "now-cast"
polls = polls.query('type == "now-cast"')
print("New shape:", polls.shape)

#### 📝 Instructor Notes - Task 6

**Key Teaching Points:**
- `query()` is cleaner than boolean indexing for simple conditions
- Must reassign result back to `polls` (or use different approach)
- The "now-cast" type represents current polling snapshots

**Common Student Errors:**
- Forgetting to reassign: `polls.query(...)` alone doesn't modify `polls`
- Wrong quotes: need double quotes inside single quotes (or vice versa)

### Task 7: Verify Rows Were Dropped (YOUR CODE)

Run the `nunique()` method again to make sure the rows have been dropped. This should show that the `adjpoll` columns have about one third as many rows as they did before.

**Expected output:** Reduced unique value counts, especially in adjpoll columns

In [ ]:
# ✅ SOLUTION - run nunique() to verify
polls.nunique()

# Note: 'type' column now shows 1 (only "now-cast" remains)
# adjpoll columns show fewer unique values

### Task 8: Drop Columns with a Single Value (YOUR CODE)

Drop all columns with a single value. These columns don't provide useful information.

**Hint:** 
- First, identify which columns have only 1 unique value from Task 7
- Use `.drop(columns=[list_of_columns])` to remove them
- Remember to reassign or use `inplace=True`

**Example syntax:**
```python
polls = polls.drop(columns=['col1', 'col2', 'col3'])
```

**Expected output:** DataFrame with fewer columns

In [ ]:
# ✅ SOLUTION - drop columns with only 1 unique value
# From nunique(), columns with 1 value: cycle, branch, type, matchup, multiversions
polls = polls.drop(columns=['cycle', 'branch', 'type', 'matchup', 'multiversions'])
print("New shape:", polls.shape)
polls.columns.tolist()

#### 📝 Instructor Notes - Task 8

**Key Teaching Points:**
- Removing constant columns reduces noise and memory
- `drop(columns=[...])` is cleaner than `drop([...], axis=1)`

**Columns to drop (all have 1 unique value after filtering):**
- `cycle` - all 2016
- `branch` - all President
- `type` - all now-cast (after filter)
- `matchup` - all same matchup
- `multiversions` - all empty

**Common Student Errors:**
- Forgetting to include all single-value columns
- Typos in column names

### Task 9: Drop Rows Where State is "U.S." (YOUR CODE)

Drop all rows where the `state` column has a value of `"U.S."`. We want to focus on state-level polling data.

**Hint:** Use `.query()` with `!=` to keep rows that are NOT "U.S."

**Example syntax:**
```python
polls = polls.query('state != "U.S."')
```

**Expected output:** DataFrame without national-level polls

In [ ]:
# ✅ SOLUTION - drop rows where state == "U.S."
polls = polls.query('state != "U.S."')
print("New shape:", polls.shape)

### Task 10: Verify with nunique() Again (YOUR CODE)

Run the `nunique()` method one more time to see how the values have changed.

**Expected output:** Updated unique value counts reflecting the cleaned data

In [ ]:
# ✅ SOLUTION - run nunique() to see updated values
polls.nunique()

---

## Part 3: Rename Columns

Make column names more user-friendly.

### Task 11: Rename the rawpoll Columns (YOUR CODE)

Rename each `rawpoll_name` column to `name_pct` where *name* is the candidate's name. For example, rename the `rawpoll_clinton` column to `clinton_pct`.

**Hint:** Use the `.rename()` method with a dictionary mapping old names to new names.

**Example syntax:**
```python
polls = polls.rename(columns={
    'old_name1': 'new_name1',
    'old_name2': 'new_name2'
})
```

**Columns to rename:**
- `rawpoll_clinton` → `clinton_pct`
- `rawpoll_trump` → `trump_pct`
- `rawpoll_johnson` → `johnson_pct`
- `rawpoll_mcmullin` → `mcmullin_pct`

**Expected output:** DataFrame with renamed columns

In [ ]:
# ✅ SOLUTION - rename the rawpoll columns
polls = polls.rename(columns={
    'rawpoll_clinton': 'clinton_pct',
    'rawpoll_trump': 'trump_pct',
    'rawpoll_johnson': 'johnson_pct',
    'rawpoll_mcmullin': 'mcmullin_pct'
})
polls.columns.tolist()

#### 📝 Instructor Notes - Task 11

**Key Teaching Points:**
- `rename(columns={...})` takes a dictionary of old→new mappings
- Only columns in the dictionary are renamed; others unchanged
- Good naming conventions make data easier to work with

**Common Student Errors:**
- Forgetting to reassign the result
- Typos in column names (case-sensitive!)

---

## Part 4: Fix Data Types

Convert columns to appropriate data types for analysis.

### Task 12: Check Current Data Types (YOUR CODE)

Run the `info()` method on the data to see the current data types.

**Expected output:** Notice that date columns are showing as `object` (string) type

In [ ]:
# ✅ SOLUTION - run info() to check data types
polls.info()

# Note: forecastdate, startdate, enddate, createddate are all 'object' (string)
# They should be datetime for proper date operations

### Task 13: Create a List of Datetime Columns (YOUR CODE)

Create a list of columns that should use the datetime type. Look at the column names - which ones contain dates?

**Hint:** Look for columns with "date" in the name.

**Example syntax:**
```python
date_cols = ['col1', 'col2', 'col3']
```

**Expected output:** A list variable containing the names of date columns

In [ ]:
# ✅ SOLUTION - create a list of datetime column names
date_cols = ['forecastdate', 'startdate', 'enddate', 'createddate']
print(date_cols)

#### 📝 Instructor Notes - Task 13

**Date columns in this dataset:**
- `forecastdate` - date of forecast
- `startdate` - poll start date
- `enddate` - poll end date
- `createddate` - record creation date

**Note:** `timestamp` also contains date/time but in a different format

### Task 14: Convert to Datetime (YOUR CODE)

Apply the Pandas `to_datetime()` method to the columns in the list of datetime columns to convert these columns to the datetime type.

**Hint:** Use a for loop to convert each column.

**Example syntax:**
```python
for col in date_cols:
    polls[col] = pd.to_datetime(polls[col])
```

**Expected output:** Date columns converted to datetime64 type

In [ ]:
# ✅ SOLUTION - convert date columns to datetime
for col in date_cols:
    polls[col] = pd.to_datetime(polls[col])

print("Conversion complete!")

#### 📝 Instructor Notes - Task 14

**Key Teaching Points:**
- `pd.to_datetime()` is smart about parsing various date formats
- Loop approach is cleaner than writing 4 separate lines
- Datetime type enables date math, filtering by date ranges, etc.

**Alternative approaches:**
```python
# Using apply
polls[date_cols] = polls[date_cols].apply(pd.to_datetime)
```

### Task 15: Convert to Category Type (YOUR CODE)

Convert the `state` and `population` columns to the category type. This is more memory-efficient for columns with a limited number of unique values.

**Hint:** Use `.astype('category')` to convert.

**Example syntax:**
```python
polls['column'] = polls['column'].astype('category')
```

**Expected output:** state and population columns converted to category type

In [ ]:
# ✅ SOLUTION - convert state and population to category type
polls['state'] = polls['state'].astype('category')
polls['population'] = polls['population'].astype('category')

print("Conversion complete!")

#### 📝 Instructor Notes - Task 15

**Key Teaching Points:**
- Category type is memory-efficient for columns with limited unique values
- `state` has ~50 values, `population` has few values (lv, rv, a, etc.)
- Categories also enable special sorting and grouping operations

**Population codes meaning:**
- `lv` = likely voters
- `rv` = registered voters
- `a` = all adults

### Task 16: Verify Data Types (YOUR CODE)

Run the `info()` method one more time to see the new data types.

**Expected output:** Date columns showing as datetime64, state and population showing as category

In [ ]:
# ✅ SOLUTION - run info() to verify new data types
polls.info()

# Note: 
# - forecastdate, startdate, enddate, createddate now show datetime64[ns]
# - state and population now show category

### Task 17: Display the Cleaned Data (YOUR CODE)

Display the Polls data to see how it looks after all the cleaning.

**Expected output:** A clean, well-organized DataFrame ready for analysis

In [ ]:
# ✅ SOLUTION - display the cleaned polls DataFrame
polls.head(10)

---

## Summary

In this exercise, you practiced cleaning data:

**Examining Data:**
- `info()` - View column types and missing values
- `nunique()` - Count unique values per column

**Dropping Rows and Columns:**
- `query()` - Filter rows based on conditions
- `drop(columns=[...])` - Remove unnecessary columns

**Renaming Columns:**
- `rename(columns={...})` - Change column names

**Fixing Data Types:**
- `pd.to_datetime()` - Convert strings to datetime
- `astype('category')` - Convert to category type

---

**Submission:** Save this notebook and submit to Canvas before the deadline.